In [29]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jan  4 03:31:19 2026

@author: mira
"""

import torch
import torch.nn as nn
from torch import optim
import os
import time
import warnings
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import pickle
from sys import exit
import math
import pprint


#!pip install finlab

import finlab

finlab.login('lSos+ax2S4Bz363QMrDmahsDZf+ioMEhRR5VAnx5bMDZt/gIfVtZ6NGRWcTlRuE9#vip_m')
from finlab import data

data.use_local_data_only = False


輸入成功!


In [30]:
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
# !tar -xzf ta-lib-0.4.0-src.tar.gz
# !cd ta-lib
# !./configure --prefix=/usr
# !make
# !sudo make install
# !pip install TA-Lib

In [31]:
# import talib

In [32]:

def filter_volatile_stocks(df, window_days=30, threshold_pct=0.30, min_gap_days=15):
    """
    Filter stocks with volatility, avoiding overlapping windows

    Parameters:
    -----------
    min_gap_days : int
        Minimum days between reported events to avoid overlap (default 15)
    """
    stock_info = {}
    selected_stocks = []

    # Ensure datetime index
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)

    # Process each stock
    for stock in df.columns:
        # Skip if column has too many NaN values
        if df[stock].isna().sum() > len(df) * 0.5:
            continue

        # Calculate rolling min and max for the window
        rolling_min = df[stock].rolling(window=f'{window_days}D', min_periods=1).min()
        rolling_max = df[stock].rolling(window=f'{window_days}D', min_periods=1).max()

        # Calculate percentage range: (max - min) / min
        with np.errstate(divide='ignore', invalid='ignore'):
            pct_range = (rolling_max - rolling_min) / rolling_min
            pct_range = pct_range.replace([np.inf, -np.inf], np.nan)

        # Find ALL dates where threshold is exceeded
        exceeds_threshold = pct_range[pct_range > threshold_pct]

        if len(exceeds_threshold) > 0:
            selected_stocks.append(stock)

            # Store dates we've already added for this stock
            added_dates = []

            # Sort by value to get highest peaks first
            exceeds_threshold_sorted = exceeds_threshold.sort_values(ascending=False)

            for date_idx in exceeds_threshold_sorted.index:
                # Check if this date is too close to any previously added date
                too_close = False
                for prev_date in added_dates:
                    # Convert to datetime if needed and calculate day difference
                    days_diff = abs((date_idx - prev_date).days)
                    if days_diff < min_gap_days:
                        too_close = True
                        break

                if too_close:
                    continue

                window_start = date_idx - pd.Timedelta(days=window_days)
                window_data = df[stock][window_start:date_idx]

                # Skip if window data is empty
                if len(window_data) == 0:
                    continue

                key = f"{stock} @ {window_start}"
                stock_info[key] = {
                    'name': stock,
                    'max_range_pct': exceeds_threshold_sorted[date_idx] * 100,
                    'date': date_idx,
                    'min_price': window_data.min(),
                    'max_price': window_data.max()
                }

                added_dates.append(date_idx)

    return stock_info


In [33]:

def filter_four_digit_stocks_simple(df):
    """Select columns where name is exactly 4 digits"""
    four_digit_stocks = []

    for col in df.columns:
        # Check if column name is 4 characters and all are digits
        if len(col) == 4 and col.isdigit():
            four_digit_stocks.append(col)

    return df[four_digit_stocks]


def filter_small_vol(df, lastday_df):

    stock_list = []

    for col in df.columns:
        vol = lastday_df[col].mean()

        if vol >= 1e6:
            stock_list.append(col)
        else:
            continue

    return df[stock_list]

def get_95_percentile(df):
  '''select 95% of predefined active stock'''
  name95 = [np.str_('6239'), np.str_('6153'), np.str_('6243'), np.str_('8464'), np.str_('3532'), np.str_('2404'), np.str_('6715'), np.str_('4915'), np.str_('6205'), np.str_('3002'), np.str_('6994'), np.str_('6515'), np.str_('2731'), np.str_('8222'), np.str_('3032'), np.str_('6706'), np.str_('6112'), np.str_('2387'), np.str_('3035'), np.str_('2258'), np.str_('1718'), np.str_('6534'), np.str_('2029'), np.str_('2013'), np.str_('2614'), np.str_('3605'), np.str_('2368'), np.str_('3694'), np.str_('6230'), np.str_('1810'), np.str_('3023'), np.str_('8996'), np.str_('1702'), np.str_('6128'), np.str_('2484'), np.str_('2491'), np.str_('2486'), np.str_('8081'), np.str_('2545'), np.str_('6504'), np.str_('3006'), np.str_('2630'), np.str_('2414'), np.str_('4934'), np.str_('6183'), np.str_('3013'), np.str_('1256'), np.str_('1752'), np.str_('5522'), np.str_('8482'), np.str_('2382'), np.str_('1402'), np.str_('6412'), np.str_('3257'), np.str_('1515'), np.str_('6807'), np.str_('3715'), np.str_('9902'), np.str_('1612'), np.str_('2002'), np.str_('4960'), np.str_('2546'), np.str_('3443'), np.str_('8028'), np.str_('6669'), np.str_('2231'), np.str_('1733'), np.str_('1527'), np.str_('2025'), np.str_('3044'), np.str_('3042'), np.str_('2406'), np.str_('3003'), np.str_('3149'), np.str_('2505'), np.str_('6776'), np.str_('2327'), np.str_('2015'), np.str_('6216'), np.str_('5871'), np.str_('2855'), np.str_('6909'), np.str_('5388'), np.str_('1309'), np.str_('2543'), np.str_('2022'), np.str_('5244'), np.str_('9105'), np.str_('1582'), np.str_('3041'), np.str_('2031'), np.str_('2028'), np.str_('2882'), np.str_('4540'), np.str_('6861'), np.str_('5285'), np.str_('9938'), np.str_('4571'), np.str_('3596'), np.str_('4551'), np.str_('9914'), np.str_('4755'), np.str_('1617'), np.str_('9934'), np.str_('1709'), np.str_('6214'), np.str_('8404'), np.str_('7740'), np.str_('5288'), np.str_('6799'), np.str_('2344'), np.str_('5608'), np.str_('8163'), np.str_('9919'), np.str_('1473'), np.str_('2069'), np.str_('6689'), np.str_('1517'), np.str_('6625'), np.str_('2615'), np.str_('2454'), np.str_('3419'), np.str_('3450'), np.str_('5284'), np.str_('2504'), np.str_('3665'), np.str_('2032'), np.str_('6531'), np.str_('1319'), np.str_('3708'), np.str_('2636'), np.str_('2481'), np.str_('6691'), np.str_('1514'), np.str_('3046'), np.str_('2324'), np.str_('6139'), np.str_('4938'), np.str_('6141'), np.str_('3296'), np.str_('6756'), np.str_('3380'), np.str_('4588'), np.str_('2038'), np.str_('2415'), np.str_('6443'), np.str_('6585'), np.str_('3454'), np.str_('6923'), np.str_('2356'), np.str_('5515'), np.str_('3591'), np.str_('5434'), np.str_('4935'), np.str_('2884'), np.str_('9921'), np.str_('1806'), np.str_('2385'), np.str_('3312'), np.str_('3711'), np.str_('5225'), np.str_('8215'), np.str_('1409'), np.str_('2547'), np.str_('2520'), np.str_('8016'), np.str_('6213'), np.str_('2348'), np.str_('6177'), np.str_('6142'), np.str_('2108'), np.str_('1225'), np.str_('6456'), np.str_('6005'), np.str_('2637'), np.str_('6414'), np.str_('3501'), np.str_('1476'), np.str_('3305'), np.str_('6525'), np.str_('3028'), np.str_('2603'), np.str_('6191'), np.str_('6533'), np.str_('2301'), np.str_('1215'), np.str_('4569'), np.str_('2227'), np.str_('2305'), np.str_('2330'), np.str_('5292'), np.str_('8476'), np.str_('3027'), np.str_('9946'), np.str_('2332'), np.str_('2399'), np.str_('6698'), np.str_('7722'), np.str_('6152'), np.str_('2408'), np.str_('6116'), np.str_('2405'), np.str_('6592'), np.str_('2462'), np.str_('3661'), np.str_('1529'), np.str_('4956'), np.str_('7799'), np.str_('7749'), np.str_('2436'), np.str_('2472'), np.str_('6873'), np.str_('6805'), np.str_('6944'), np.str_('3189'), np.str_('1504'), np.str_('4526'), np.str_('4572'), np.str_('2497'), np.str_('3052'), np.str_('9945'), np.str_('5222'), np.str_('6782'), np.str_('2308'), np.str_('3135'), np.str_('2374'), np.str_('6164'), np.str_('6589'), np.str_('2887'), np.str_('9904'), np.str_('3652'), np.str_('2233'), np.str_('1597'), np.str_('6949'), np.str_('2364'), np.str_('2444'), np.str_('2347'), np.str_('2748'), np.str_('2376'), np.str_('7788'), np.str_('3550'), np.str_('4763'), np.str_('1611'), np.str_('9958'), np.str_('3209'), np.str_('1909'), np.str_('2027'), np.str_('8045'), np.str_('4737'), np.str_('2605'), np.str_('6796'), np.str_('1463'), np.str_('6166'), np.str_('2442'), np.str_('1723'), np.str_('2515'), np.str_('2607'), np.str_('4133'), np.str_('4912'), np.str_('2393'), np.str_('6757'), np.str_('4770'), np.str_('2528'), np.str_('3607'), np.str_('5243'), np.str_('4555'), np.str_('3024'), np.str_('2388'), np.str_('6605'), np.str_('2329'), np.str_('4119'), np.str_('6505'), np.str_('1308'), np.str_('6283'), np.str_('2482'), np.str_('4771'), np.str_('2488'), np.str_('3557'), np.str_('8499'), np.str_('3576'), np.str_('2890'), np.str_('6753'), np.str_('3530'), np.str_('4585'), np.str_('6591'), np.str_('3533'), np.str_('4720'), np.str_('1305'), np.str_('5607'), np.str_('1905'), np.str_('3543'), np.str_('2211'), np.str_('2453'), np.str_('2204'), np.str_('1783'), np.str_('6202'), np.str_('6928'), np.str_('3055'), np.str_('6438'), np.str_('4952'), np.str_('8114'), np.str_('1560'), np.str_('4746'), np.str_('3703'), np.str_('3437'), np.str_('3048'), np.str_('6224'), np.str_('6415'), np.str_('3535'), np.str_('4576'), np.str_('2539'), np.str_('2340'), np.str_('3406'), np.str_('6477'), np.str_('2495'), np.str_('6862'), np.str_('9935'), np.str_('2390'), np.str_('2062'), np.str_('3037'), np.str_('1605'), np.str_('3022'), np.str_('8454'), np.str_('6449'), np.str_('1314'), np.str_('1472'), np.str_('2328'), np.str_('1802'), np.str_('3583'), np.str_('6965'), np.str_('2103'), np.str_('3338'), np.str_('2377'), np.str_('2832'), np.str_('2342'), np.str_('1307'), np.str_('7610'), np.str_('2323'), np.str_('2527'), np.str_('1903'), np.str_('8021'), np.str_('4766'), np.str_('6916'), np.str_('3653'), np.str_('6768'), np.str_('6215'), np.str_('3062'), np.str_('6657'), np.str_('6781'), np.str_('7721'), np.str_('6442'), np.str_('3036'), np.str_('2413'), np.str_('2439'), np.str_('3311'), np.str_('3518'), np.str_('1532'), np.str_('2530'), np.str_('6789'), np.str_('1303'), np.str_('3029'), np.str_('2501'), np.str_('2409'), np.str_('3050'), np.str_('1449'), np.str_('1817'), np.str_('1710'), np.str_('8478'), np.str_('6869'), np.str_('6133'), np.str_('2367'), np.str_('5521'), np.str_('8438'), np.str_('8150'), np.str_('4557'), np.str_('4564'), np.str_('2867'), np.str_('2611'), np.str_('1540'), np.str_('3592'), np.str_('2429'), np.str_('2425'), np.str_('8131'), np.str_('2704'), np.str_('3701'), np.str_('2492'), np.str_('6526'), np.str_('8103'), np.str_('8110'), np.str_('5283'), np.str_('1717'), np.str_('8271'), np.str_('2423'), np.str_('2461'), np.str_('8261'), np.str_('6674'), np.str_('8422'), np.str_('3034'), np.str_('2383'), np.str_('1608'), np.str_('1451'), np.str_('2537'), np.str_('4977'), np.str_('3231'), np.str_('3504'), np.str_('2354'), np.str_('1609'), np.str_('2885'), np.str_('1316'), np.str_('1563'), np.str_('1708'), np.str_('1312'), np.str_('9907'), np.str_('2753'), np.str_('1721'), np.str_('3563'), np.str_('1519'), np.str_('3030'), np.str_('1438'), np.str_('3712'), np.str_('1459'), np.str_('4916'), np.str_('2033'), np.str_('8213'), np.str_('1528'), np.str_('6933'), np.str_('6472'), np.str_('2312'), np.str_('6269'), np.str_('2359'), np.str_('2355'), np.str_('2369'), np.str_('2493'), np.str_('1618'), np.str_('6491'), np.str_('2349'), np.str_('6288'), np.str_('3025'), np.str_('3031'), np.str_('4968'), np.str_('3356'), np.str_('6550'), np.str_('8039'), np.str_('2104'), np.str_('5531'), np.str_('9927'), np.str_('2438'), np.str_('6206'), np.str_('3017'), np.str_('3416'), np.str_('2889'), np.str_('2466'), np.str_('4976'), np.str_('6937'), np.str_('8374'), np.str_('2524'), np.str_('2345'), np.str_('4562'), np.str_('1773'), np.str_('1326'), np.str_('6117'), np.str_('2460'), np.str_('1101'), np.str_('6863'), np.str_('1219'), np.str_('8070'), np.str_('2913'), np.str_('2440'), np.str_('8473'), np.str_('6834'), np.str_('6890'), np.str_('4958'), np.str_('5306'), np.str_('6743'), np.str_('6235'), np.str_('2441'), np.str_('2105'), np.str_('4438'), np.str_('2395'), np.str_('2929'), np.str_('3060'), np.str_('6108'), np.str_('1477'), np.str_('8462'), np.str_('6668'), np.str_('4919'), np.str_('1325'), np.str_('1808'), np.str_('3545'), np.str_('6120'), np.str_('1310'), np.str_('2509'), np.str_('2451'), np.str_('5258'), np.str_('6189'), np.str_('2915'), np.str_('7705'), np.str_('2457'), np.str_('2609'), np.str_('3005'), np.str_('3716'), np.str_('6670'), np.str_('2208'), np.str_('2419'), np.str_('2102'), np.str_('2351'), np.str_('2338'), np.str_('2363'), np.str_('1732'), np.str_('2365'), np.str_('2243'), np.str_('5469'), np.str_('6830'), np.str_('4722'), np.str_('6918'), np.str_('6887'), np.str_('2485'), np.str_('1444'), np.str_('6282'), np.str_('2241'), np.str_('6541'), np.str_('2601'), np.str_('5234'), np.str_('0052'), np.str_('2014'), np.str_('1440'), np.str_('2905'), np.str_('3016'), np.str_('2723'), np.str_('1589'), np.str_('6582'), np.str_('6770'), np.str_('4949'), np.str_('8046'), np.str_('1220'), np.str_('2030'), np.str_('1446'), np.str_('4536'), np.str_('6155'), np.str_('2476'), np.str_('9910'), np.str_('8033'), np.str_('2464'), np.str_('2331'), np.str_('3011'), np.str_('4583'), np.str_('2645'), np.str_('2314'), np.str_('3686'), np.str_('6165'), np.str_('4566'), np.str_('1464'), np.str_('3051'), np.str_('6919'), np.str_('6285'), np.str_('6806'), np.str_('3057'), np.str_('3004'), np.str_('2371'), np.str_('6197'), np.str_('3058'), np.str_('1616'), np.str_('5519'), np.str_('2478'), np.str_('6742'), np.str_('2722'), np.str_('4532'), np.str_('3019'), np.str_('2449'), np.str_('6176'), np.str_('3588'), np.str_('2254'), np.str_('2362'), np.str_('3021'), np.str_('2634'), np.str_('1786'), np.str_('1522'), np.str_('6405'), np.str_('3702'), np.str_('6257'), np.str_('2236'), np.str_('2397'), np.str_('2337'), np.str_('6838'), np.str_('2357'), np.str_('3043'), np.str_('2352'), np.str_('1231'), np.str_('2610'), np.str_('3059'), np.str_('9955'), np.str_('4545'), np.str_('1539'), np.str_('6901'), np.str_('3056'), np.str_('6906'), np.str_('1809'), np.str_('2455'), np.str_('8105'), np.str_('6024'), np.str_('6416'), np.str_('1447'), np.str_('6426'), np.str_('1760'), np.str_('4764'), np.str_('3447'), np.str_('8072'), np.str_('2613'), np.str_('2428'), np.str_('6672'), np.str_('1414'), np.str_('1727'), np.str_('2467'), np.str_('5203'), np.str_('1590'), np.str_('1434'), np.str_('1603'), np.str_('8112'), np.str_('1466'), np.str_('0050'), np.str_('2353'), np.str_('6962'), np.str_('2458'), np.str_('4961'), np.str_('6409'), np.str_('6902'), np.str_('3706'), np.str_('3010'), np.str_('8011'), np.str_('2612'), np.str_('2489'), np.str_('3094'), np.str_('2739'), np.str_('6854'), np.str_('9941'), np.str_('3015'), np.str_('2049'), np.str_('2420'), np.str_('3049'), np.str_('2239'), np.str_('2059'), np.str_('2375'), np.str_('6209'), np.str_('3014'), np.str_('1301'), np.str_('6446'), np.str_('6957'), np.str_('2417'), np.str_('4414'), np.str_('2471'), np.str_('6431'), np.str_('3617'), np.str_('9933'), np.str_('4906'), np.str_('1598'), np.str_('2017'), np.str_('6271'), np.str_('1460'), np.str_('1419'), np.str_('8104'), np.str_('6558'), np.str_('3645'), np.str_('2534'), np.str_('4989'), np.str_('2402'), np.str_('6885'), np.str_('2540'), np.str_('1533'), np.str_('2034'), np.str_('8249'), np.str_('2020'), np.str_('3038'), np.str_('2421'), np.str_('1795'), np.str_('2360'), np.str_('1313'), np.str_('5907'), np.str_('2250'), np.str_('4967'), np.str_('9939'), np.str_('2424'), np.str_('2317'), np.str_('6658'), np.str_('3138'), np.str_('6835'), np.str_('1455'), np.str_('6598'), np.str_('1442'), np.str_('1513'), np.str_('6451'), np.str_('8442'), np.str_('1436'), np.str_('3167'), np.str_('2228'), np.str_('2618'), np.str_('6278'), np.str_('2606'), np.str_('1536'), np.str_('6931'), np.str_('3026'), np.str_('1711'), np.str_('3376'), np.str_('2302'), np.str_('2392'), np.str_('3673'), np.str_('0057'), np.str_('7780'), np.str_('6831'), np.str_('1587'), np.str_('5484'), np.str_('2316'), np.str_('1524'), np.str_('2401'), np.str_('8429'), np.str_('3515'), np.str_('4142'), np.str_('9940'), np.str_('3704'), np.str_('5534'), np.str_('3090'), np.str_('3092'), np.str_('1503'), np.str_('1304'), np.str_('5269'), np.str_('3528'), np.str_('7750'), np.str_('4164'), np.str_('5471'), np.str_('6719'), np.str_('3413'), np.str_('2468'), np.str_('3040'), np.str_('6579'), np.str_('4739'), np.str_('2516'), np.str_('3008'), np.str_('4927'), np.str_('6695'), np.str_('2477'), np.str_('3679'), np.str_('2101'), np.str_('1713'), np.str_('2313'), np.str_('2106'), np.str_('2617'), np.str_('0051'), np.str_('4306'), np.str_('2891'), np.str_('3714'), np.str_('6168'), np.str_('2762'), np.str_('2201'), np.str_('2465'), np.str_('5706'), np.str_('1342'), np.str_('4560'), np.str_('6792'), np.str_('2498'), np.str_('3346'), np.str_('1432'), np.str_('2427'), np.str_('3047'), np.str_('9906'), np.str_('2431'), np.str_('1789'), np.str_('2426'), np.str_('3481'), np.str_('2009'), np.str_('2480'), np.str_('2538'), np.str_('1736'), np.str_('8467'), np.str_('6196'), np.str_('9802'), np.str_('3622'), np.str_('8210'), np.str_('7736')]

  return df[name95]



In [34]:


data.set_storage(data.FileStorage(path='finlabdata'))
start_date_str = '2020-01-01'
end_date_str = '2025-05-31'

data.truncate_start = start_date_str
data.truncate_end = end_date_str


In [35]:
# start_date = datetime.strptime("2020-01-01", "%Y-%m-%d")
# end_date = datetime.strptime("2025-05-31", "%Y-%m-%d")#start_date = datetime.strptime("2020-01-01", "%Y-%m-%d")
# #end_date = datetime.strptime("2025-12-31", "%Y-%m-%d")

target_index = pd.date_range(
    start=start_date_str,
    end=end_date_str,
    freq="D"
)


In [36]:
from copy import deepcopy
df1 = data.get('price:成交筆數')
df1_backup = deepcopy(df1)
df1 = df1 + 1
df1 = np.log(df1)
df2 = data.get('price:成交金額')
df2 = df2 + 1
df2 = np.log(df2)
df3 = data.get('etl:adj_close')

# df5 = pd.DataFrame(columns=df3.columns, index=df3.index)
# df6 = pd.DataFrame(columns=df3.columns, index=df3.index)
# # df5 = talib.RSI(df3, timeperiod=14)
# # df6 = talib.MOM(df3, timeperiod=10)
# for col in df3.columns:
#   df5[col] = talib.RSI(df3[col], timeperiod=14)
# for col in df3.columns:
#   df6[col] = talib.MOM(df3[col], timeperiod=10)

df7 = data.get('institutional_investors_trading_summary:外資自營商買賣超股數')
df8 = data.get('institutional_investors_trading_summary:投信買賣超股數')
df9 = data.get('institutional_investors_trading_summary:自營商買賣超股數(自行買賣)')
print(np.isfinite(df7.values).all())
print(np.isfinite(df8.values).all())
print(np.isfinite(df9.values).all())


df7 = df7/1000 + 1
df8 = df8/1000 + 1
df9 = df9/1000 + 1


Daily usage: 496.6 / 5000 MB - etl:adj_close
False
False
False


In [37]:

list_of_dfs = [df1, df2, df3, df7, df8, df9]
for df in list_of_dfs:
  i = ~np.isfinite(df)
  df[i] = 0

for df in list_of_dfs:
  print(np.isfinite(df.values).all())


True
True
True
True
True
True


# select only four-digit tickers & discard small volume (張)

In [38]:

# select volume
df1_fd = filter_four_digit_stocks_simple(deepcopy(df1_backup))

tickers = df1_fd.columns
mean_list = []
for ticker in tickers:
    mean_no_zeros = df1_fd[df1_fd[ticker] != 0][ticker].mean()
    mean_list.append((mean_no_zeros,ticker))

mean_list = sorted(mean_list, key = lambda mean_: mean_[0])
# Decide by volume
print(len(mean_list))
subset = [item[1] for item in mean_list if item[0] > 1000]
print(len(subset))

def filter(df, subset):
    return df[subset]


2209
667


In [39]:

data_filtered = []
for df in list_of_dfs:
    #select95 = get_95_percentile(df)
    df = filter(df, subset)
    data_filtered.append(df)
print(len(data_filtered))
print(data_filtered[0].shape)
print(data_filtered[0].columns)
print(type(list(data_filtered[0].columns)))
ticker_index_dict = {}
for index,ticker in enumerate(list(data_filtered[0].columns)):
    ticker_index_dict[index] = ticker
print(ticker_index_dict)
with open('ticker_index_dict.pkl', 'wb') as f:
    pickle.dump(ticker_index_dict, f)

6
(1311, 667)
Index(['1434', '1447', '1444', '1409', '1440', '1326', '1402', '1102', '0050',
       '0056',
       ...
       '9907', '4526', '8436', '4533', '4510', '8299', '9904', '8358', '9958',
       '9945'],
      dtype='object', name='symbol', length=667)
<class 'list'>
{0: '1434', 1: '1447', 2: '1444', 3: '1409', 4: '1440', 5: '1326', 6: '1402', 7: '1102', 8: '0050', 9: '0056', 10: '1217', 11: '1216', 12: '1101', 13: '1310', 14: '1229', 15: '1210', 16: '1325', 17: '1313', 18: '1309', 19: '1316', 20: '1308', 21: '1305', 22: '1312', 23: '1319', 24: '1304', 25: '1303', 26: '1301', 27: '1314', 28: '1455', 29: '1476', 30: '1515', 31: '1477', 32: '1503', 33: '1504', 34: '1514', 35: '1519', 36: '1522', 37: '1528', 38: '1540', 39: '1569', 40: '1529', 41: '1584', 42: '1536', 43: '1524', 44: '1560', 45: '1597', 46: '1701', 47: '1709', 48: '1710', 49: '1611', 50: '1612', 51: '1618', 52: '1608', 53: '1609', 54: '1605', 55: '1712', 56: '1736', 57: '1762', 58: '1732', 59: '1734', 60: '1714',

In [40]:

processed_dfs_list = []

#for index, df  in enumerate(fourdigitfiltered):
for index, df  in enumerate(data_filtered):
    #missedfourdigit = list(set(fourdigitfiltered[0].columns) - set(fourdigitfiltered[index].columns))
    df_ = deepcopy(df)
    #for col_name in missedfourdigit:
    #  df_[''+str(col_name)] = 0
    print('='*60)
    #print(missedfourdigit)
    print('='*60)
    print(df_.shape)
    print(df_.columns)
    #print(df.tail())

    df_.index = (
        pd.to_datetime(df_.index)
        .tz_localize(None)
        .normalize()
    )

    df_ = (
        df_
        .reindex(target_index)
        .interpolate(method="time")
        .ffill()
        .bfill()
    )

    df_.fillna(0)
    df_.replace(np.nan, 0)
    df_.replace(np.inf, 100)
    df_.replace(-np.inf, -100)

    processed_dfs_list.append(df_)
    print(type(df_))


processed_dfs = np.array(list(map(lambda df: df.to_numpy(), processed_dfs_list)))
print(processed_dfs.shape)
print(processed_dfs[0].shape)



(1311, 667)
Index(['1434', '1447', '1444', '1409', '1440', '1326', '1402', '1102', '0050',
       '0056',
       ...
       '9907', '4526', '8436', '4533', '4510', '8299', '9904', '8358', '9958',
       '9945'],
      dtype='object', name='symbol', length=667)
<class 'finlab.dataframe.FinlabDataFrame'>
(1311, 667)
Index(['1434', '1447', '1444', '1409', '1440', '1326', '1402', '1102', '0050',
       '0056',
       ...
       '9907', '4526', '8436', '4533', '4510', '8299', '9904', '8358', '9958',
       '9945'],
      dtype='object', name='symbol', length=667)
<class 'finlab.dataframe.FinlabDataFrame'>
(1311, 667)
Index(['1434', '1447', '1444', '1409', '1440', '1326', '1402', '1102', '0050',
       '0056',
       ...
       '9907', '4526', '8436', '4533', '4510', '8299', '9904', '8358', '9958',
       '9945'],
      dtype='object', name='symbol', length=667)
<class 'finlab.dataframe.FinlabDataFrame'>
(1311, 667)
Index(['1434', '1447', '1444', '1409', '1440', '1326', '1402', '1102', '0050

In [41]:
processed_dfs = np.transpose(processed_dfs, (1, 0, 2))
print(processed_dfs.shape)
#np.save('20200101-20251231-95perT.npy', processed_dfs)


np.save(start_date_str+'-'+end_date_str+'-637'+'.npy', processed_dfs)
#np.save(start_date_str+'-'+end_date_str+'-95perT'+'.npy', processed_dfs)

(1978, 6, 667)


In [42]:
# np.save('20200101-20251231-95per.npy', processed_dfs)
# from google.colab import files
# files.download('20200101-20251231-95per.npy')

In [43]:
print(df2.columns)

Index(['0015', '0050', '0051', '0052', '0053', '0054', '0055', '0056', '0057',
       '0058',
       ...
       '9944', '9945', '9946', '9949', '9950', '9951', '9955', '9958', '9960',
       '9962'],
      dtype='object', name='symbol', length=2709)


In [44]:
selected_stock_names = ['2330', '2317', '2359', '3450', '3081', '2609', '3231', '3706', '6533']
selected_stock = [df1.columns.get_loc(col) for col in selected_stock_names]

selected_stock_names2 = ['6285','3443','8299','2360']
selected_stock2 = [df1.columns.get_loc(col) for col in selected_stock_names2]



In [45]:
print(selected_stock)
print(selected_stock2)


[799, 790, 823, 1338, 1194, 983, 1255, 1475, 2157]
[2080, 1335, 2535, 824]
